In [1]:
from datasets import load_from_disk
from utils import convert_batch_to_ids, prepare_tokenizer
from torch.utils.data import DataLoader

In [2]:
tkn, _ = prepare_tokenizer('./tokenizer/')
ds = load_from_disk('/root/autodl-tmp/ultrachat/')

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['conversation_id', 'dataset', 'conversation'],
        num_rows: 1468338
    })
})

In [4]:
ds = ds.remove_columns(['conversation_id', 'dataset'])

In [5]:
def process_line(line):
    conv_list = line['conversation']
    out_list = []
    for conv in conv_list:
        out_list.append(conv['human'])
        out_list.append(conv['assistant'])
    out_list.append('')
    eos_sep = f' {tkn.eos_token}'
    return {'text': eos_sep.join(out_list)}

ds_combined = ds.map(process_line, num_proc=15).remove_columns('conversation')

Map (num_proc=15):   0%|          | 0/1468338 [00:00<?, ? examples/s]

In [6]:
ds_combined

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1468338
    })
})

In [7]:
ds_combined['train'][0]

{'text': "How can cross training benefit groups like runners, swimmers, or weightlifters? </s>Cross training can benefit groups like runners, swimmers, or weightlifters in the following ways:\n\n1. Reduces the risk of injury: Cross training involves different types of exercises that work different muscle groups. This reduces the risk of overuse injuries that may result from repetitive use of the same muscles.\n\n2. Improves overall fitness: Cross training helps improve overall fitness levels by maintaining a balance of strength, endurance, flexibility, and cardiovascular fitness.\n\n3. Breaks monotony: Cross training adds variety to your fitness routine by introducing new exercises, which can help you stay motivated and avoid boredom that often comes with doing the same exercises repeatedly.\n\n4. Increases strength: Cross training helps in building strength by incorporating exercises that target different muscle groups. This helps you build strength in areas that may be underdeveloped

In [8]:
ds_combined.save_to_disk('/root/autodl-tmp/uc_parsed')

Saving the dataset (0/19 shards):   0%|          | 0/1468338 [00:00<?, ? examples/s]